##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Trabalhando com tensores esparsos

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/sparse_tensor"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/sparse_tensor_guide.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/sparse_tensor_guide.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/sparse_tensor_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Ao trabalhar com tensores que contêm muitos valores zero, é importante armazená-los de maneira eficiente em termos de espaço e tempo. Os tensores esparsos permitem o armazenamento e o processamento eficiente de tensores que contêm muitos valores zero. Os tensores esparsos são amplamente usados em esquemas de codificação como [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) para o pré-processamento de dados em aplicativos de PLN e de imagens com muitos pixels escuros em aplicativos de visão computacional.

## Tensores esparsos no TensorFlow

O TensorFlow representa tensores esparsos pelo objeto `tf.sparse.SparseTensor`. No momento, os tensores esparsos no TensorFlow são codificados usando o formato de lista de coordenadas (COO). Esse formato de codificação é otimizado para matrizes hiperesparsas como embeddings.

A codificação COO para tensores esparsos é composta de:

- `values`: um tensor 1D com formato `[N]`, que contém todos os valores diferentes de zero.
- `indices`: um tensor 2D com formato `[N, rank]`, que contém os índices de todos os valores diferentes de zero.
- `dense_shape`: um tensor 1D com formato `[rank]`, especificando o formato do tensor.

Um valor ***diferente de zero*** no contexto de um `tf.sparse.SparseTensor` é um valor que não é explicitamente codificado. É possível incluir valores zero explicitamente nos `valores` de uma matriz esparsa de COO, mas esses "zeros explícitos" geralmente não são incluídos quando se referem a valores diferentes de zero em um tensor esparso.

Observação: `tf.sparse.SparseTensor` não exige que os índices/valores estejam em qualquer ordem específica, mas várias ops presumem que eles são orientados por linha. Use `tf.sparse.reorder` para criar uma cópia do tensor esparso na ordem canônica orientada por linha. 

## Criando um `tf.sparse.SparseTensor`

Construa tensores esparsos ao especificar diretamente seus `values`, `indices` e `dense_shape`.

In [ ]:
import tensorflow as tf

In [ ]:
st1 = tf.sparse.SparseTensor(indices=[[0, 3], [2, 4]],
                      values=[10, 20],
                      dense_shape=[3, 10])

<img src="images/sparse_tensor.png">

Ao usar a função `print()` para imprimir um tensor esparso, ela mostra o conteúdo dos três tensores componentes:

In [ ]:
print(st1)

É mais fácil de entender o conteúdo de um tensor esparso se os `values` diferentes de zero estiverem alinhados aos seus `indices` correspondentes. Defina uma função helper a fim de realizar o pretty-print dos tensores esparsos, para que cada valor diferente de zero apareça na sua própria linha.

In [ ]:
def pprint_sparse_tensor(st):
  s = "<SparseTensor shape=%s \n values={" % (st.dense_shape.numpy().tolist(),)
  for (index, value) in zip(st.indices, st.values):
    s += f"\n  %s: %s" % (index.numpy().tolist(), value.numpy().tolist())
  return s + "}>"

In [ ]:
print(pprint_sparse_tensor(st1))

Você também pode construir tensores esparsos a partir de tensores densos ao usar `tf.sparse.from_dense` e convertê-los de volta para tensores densos ao usar `tf.sparse.to_dense`.

In [ ]:
st2 = tf.sparse.from_dense([[1, 0, 0, 8], [0, 0, 0, 0], [0, 0, 3, 0]])
print(pprint_sparse_tensor(st2))

In [ ]:
st3 = tf.sparse.to_dense(st2)
print(st3)

## Manipulando tensores esparsos

Use os utilitários no pacote `tf.sparse` para manipular os tensores esparsos. Ops como `tf.math.add` que você pode usar para a manipulação aritmética de tensores densos não funcionam com tensores esparsos.

Adicione tensores esparsos do mesmo formato ao usar `tf.sparse.add`. 

In [ ]:
st_a = tf.sparse.SparseTensor(indices=[[0, 2], [3, 4]],
                       values=[31, 2], 
                       dense_shape=[4, 10])

st_b = tf.sparse.SparseTensor(indices=[[0, 2], [7, 0]],
                       values=[56, 38],
                       dense_shape=[4, 10])

st_sum = tf.sparse.add(st_a, st_b)

print(pprint_sparse_tensor(st_sum))

Use `tf.sparse.sparse_dense_matmul` para multiplicar os tensores esparsos com matrizes densas.

In [ ]:
st_c = tf.sparse.SparseTensor(indices=([0, 1], [1, 0], [1, 1]),
                       values=[13, 15, 17],
                       dense_shape=(2,2))

mb = tf.constant([[4], [6]])
product = tf.sparse.sparse_dense_matmul(st_c, mb)

print(product)

Coloque tensores esparsos juntos usando `tf.sparse.concat` e os separe usando `tf.sparse.slice`.


In [ ]:
sparse_pattern_A = tf.sparse.SparseTensor(indices = [[2,4], [3,3], [3,4], [4,3], [4,4], [5,4]],
                         values = [1,1,1,1,1,1],
                         dense_shape = [8,5])
sparse_pattern_B = tf.sparse.SparseTensor(indices = [[0,2], [1,1], [1,3], [2,0], [2,4], [2,5], [3,5], 
                                              [4,5], [5,0], [5,4], [5,5], [6,1], [6,3], [7,2]],
                         values = [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
                         dense_shape = [8,6])
sparse_pattern_C = tf.sparse.SparseTensor(indices = [[3,0], [4,0]],
                         values = [1,1],
                         dense_shape = [8,6])

sparse_patterns_list = [sparse_pattern_A, sparse_pattern_B, sparse_pattern_C]
sparse_pattern = tf.sparse.concat(axis=1, sp_inputs=sparse_patterns_list)
print(tf.sparse.to_dense(sparse_pattern))

In [ ]:
sparse_slice_A = tf.sparse.slice(sparse_pattern_A, start = [0,0], size = [8,5])
sparse_slice_B = tf.sparse.slice(sparse_pattern_B, start = [0,5], size = [8,6])
sparse_slice_C = tf.sparse.slice(sparse_pattern_C, start = [0,10], size = [8,6])
print(tf.sparse.to_dense(sparse_slice_A))
print(tf.sparse.to_dense(sparse_slice_B))
print(tf.sparse.to_dense(sparse_slice_C))

Se você estiver usando o TensorFlow 2.4 ou superior, use `tf.sparse.map_values` para operações elemento a elemento em valores diferentes de zero nos tensores esparsos.

In [ ]:
st2_plus_5 = tf.sparse.map_values(tf.add, st2, 5)
print(tf.sparse.to_dense(st2_plus_5))

Observe que somente os valores diferentes de zero foram modificados — os valores zero permanecem zero.

De maneira equivalente, você pode seguir o padrão de design abaixo para versões mais antigas do TensorFlow:

In [ ]:
st2_plus_5 = tf.sparse.SparseTensor(
    st2.indices,
    st2.values + 5,
    st2.dense_shape)
print(tf.sparse.to_dense(st2_plus_5))

## Usando `tf.sparse.SparseTensor` com outras APIs TensorFlow

Tensores esparsos funcionam de maneira transparente com estas APIs TensorFlow:

- `tf.keras`
- `tf.data`
- Protobuf do `tf.Train.Example`
- `tf.function`
- `tf.while_loop`
- `tf.cond`
- `tf.identity`
- `tf.cast`
- `tf.print`
- `tf.saved_model`
- `tf.io.serialize_sparse`
- `tf.io.serialize_many_sparse`
- `tf.io.deserialize_many_sparse`
- `tf.math.abs`
- `tf.math.negative`
- `tf.math.sign`
- `tf.math.square`
- `tf.math.sqrt`
- `tf.math.erf`
- `tf.math.tanh`
- `tf.math.bessel_i0e`
- `tf.math.bessel_i1e`

Confira os exemplos a seguir para algumas das APIs acima.

### `tf.keras`

Um subconjunto da API `tf.keras` é compatível com tensores esparsos sem ops caras de conversão ou casting. A API Keras permite que você passe tensores esparsos como entradas para um modelo do Keras. Defina `sparse=True` ao chamar `tf.keras.Input` ou `tf.keras.layers.InputLayer`. Você pode passar tensores esparsos entre as camadas do Keras e também fazer os modelos do Keras retornarem esses tensores como saídas. Se você usar tensores esparsos em camadas `tf.keras.layers.Dense` no seu modelo, elas gerarão tensores densos.

O exemplo abaixo mostra como passar um tensor esparso como entrada para um modelo do Keras se você só usa camadas que aceitam entradas esparsas.

In [ ]:
x = tf.keras.Input(shape=(4,), sparse=True)
y = tf.keras.layers.Dense(4)(x)
model = tf.keras.Model(x, y)

sparse_data = tf.sparse.SparseTensor(
    indices = [(0,0),(0,1),(0,2),
               (4,3),(5,0),(5,1)],
    values = [1,1,1,1,1,1],
    dense_shape = (6,4)
)

model(sparse_data)

model.predict(sparse_data)

### `tf.data`

A API `tf.data` permite criar pipelines de entrada complexos a partir de partes simples e reutilizáveis. A estrutura de dados principal é `tf.data.Dataset`, que representa uma sequência de elementos, onde cada um consiste em um ou mais componentes.

#### Criando datasets com tensores esparsos

Crie datasets a partir de tensores esparsos usando os mesmos métodos utilizados para a criação com `tf.Tensor`s ou arrays do NumPy, como `tf.data.Dataset.from_tensor_slices`. Essa op preserva a esparsidade (ou natureza esparsa) dos dados.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(sparse_data)
for element in dataset: 
  print(pprint_sparse_tensor(element))

#### Criando e separando lotes de datasets com tensores esparsos

Você pode criar lotes (combinar elementos consecutivos em um único elemento) de datasets ou separá-los com tensores esparsos usando os métodos `Dataset.batch` e `Dataset.unbatch`, respectivamente.

In [ ]:
batched_dataset = dataset.batch(2)
for element in batched_dataset:
  print (pprint_sparse_tensor(element))

In [ ]:
unbatched_dataset = batched_dataset.unbatch()
for element in unbatched_dataset:
  print (pprint_sparse_tensor(element))

Você também pode usar `tf.data.experimental.dense_to_sparse_batch` para criar lotes de elementos de dataset com diferentes formatos nos tensores esparsos. 

#### Transformando datasets com tensores esparsos

Transforme e crie tensores esparsos em datasets usando `Dataset.map`.

In [ ]:
transform_dataset = dataset.map(lambda x: x*2)
for i in transform_dataset:
  print(pprint_sparse_tensor(i))

### tf.train.Example

`tf.train.Example` é uma codificação de protobuf padrão para dados do TensorFlow. Ao usar tensores esparsos com `tf.train.Example`, você pode:

- Ler dados de comprimento variável em um `tf.sparse.SparseTensor` usando `tf.io.VarLenFeature`. No entanto, você deve considerar usar `tf.io.RaggedFeature` em vez disso.

- Ler dados esparsos arbitrários em um `tf.sparse.SparseTensor` usando `tf.io.SparseFeature`, que utiliza três chaves de características separadas para armazenar `indices`, `values` e `dense_shape`.

### `tf.function`

O decorador `tf.function` pré-computa os grafos do TensorFlow para as funções em Python, o que pode melhorar significativamente o desempenho do seu código do TensorFlow. Os tensores esparsos funcionam de maneira transparente com a `tf.function` e as [funções concretas](https://www.tensorflow.org/guide/function#obtaining_concrete_functions).

In [ ]:
@tf.function
def f(x,y):
  return tf.sparse.sparse_dense_matmul(x,y)

a = tf.sparse.SparseTensor(indices=[[0, 3], [2, 4]],
                    values=[15, 25],
                    dense_shape=[3, 10])

b = tf.sparse.to_dense(tf.sparse.transpose(a))

c = f(a,b)

print(c)

## Diferenciando valores ausentes de valores zero

A maioria das ops no `tf.sparse.SparseTensor`s tratam valores ausentes e valores zero explícitos de maneira idêntica. Isso é desde a concepção — `tf.sparse.SparseTensor` deve agir exatamente como um tensor denso.

No entanto, há alguns casos em que pode ser útil diferenciar valores zero de valores ausentes. Em especial, isso permite uma maneira de codificar dados ausentes/desconhecidos nos seus dados de treinamento. Por exemplo, considere um caso de uso em que você tem um tensor de pontuações (que podem ter qualquer valor de ponto flutuante de -Inf a +Inf), com algumas pontuações ausentes. Você pode codificar esse tensor usando um tensor esparso em que os zeros explícitos são pontuações zero conhecidas, mas os valores zero implícitos realmente representam dados ausentes, e não zero.

Observação: geralmente, esse não é o uso pretendido de `tf.sparse.SparseTensor`, e você deve também considerar outras técnicas para codificação, como usar um tensor de máscara separado que identifica as localizações de valores conhecidos/desconhecidos. No entanto, tenha cautela ao usar essa abordagem, já que a maioria das operações esparsas tratarão valores zero explícitos e implícitos de maneira idêntica.

Observe que algumas ops, como `tf.sparse.reduce_max`, não tratam valores ausentes como se fossem zero. Por exemplo, quando você executa o bloco de código abaixo, o resultado esperado é `0`. No entanto, por causa dessa exceção, o resultado é `-3`.

In [ ]:
print(tf.sparse.reduce_max(tf.sparse.from_dense([-5, 0, -3])))

Em comparação, ao aplicar `tf.math.reduce_max` a um tensor denso, a saída é 0 como esperado.

In [ ]:
print(tf.math.reduce_max([-5, 0, -3]))

## Leituras e recursos adicionais

- Consulte o [guia de tensores](https://www.tensorflow.org/guide/tensor) para saber mais sobre eles.
- Leia o [guia de tensores irregulares](https://www.tensorflow.org/guide/ragged_tensor) para saber como trabalhar com esse tipo de tensor, que permite usar dados não uniformes.
- Confira este modelo de detecção de objetos no [TensorFlow Model Garden](https://github.com/tensorflow/models) que usa tensores esparsos em um [decoder de dados `tf.Example`](https://github.com/tensorflow/models/blob/9139a7b90112562aec1d7e328593681bd410e1e7/research/object_detection/data_decoders/tf_example_decoder.py).
